#EITQ TP4: Languages

## Credits

This tutorial is built around the examples from the `treelib` and `tatsu` documentations, as well as a [regexp](https://github.com/tesla809/intro-to-python-jupyter-notebooks/blob/master/47-Regular%20Expressions.ipynb) tutorial.  

##Introduction

The tutorial covers:

*   Trees
*   Regular expressions
*   Parsing

## Python Versions


We'll be using Python 3. You can check your Python version at the command line by running `python --version`. In Colab, we can enforce the Python version by clicking `Runtime -> Change Runtime Type` and selecting `python3`.

In [4]:
!python --version

Python 3.10.8


## Trees

There are many ways to organize your data in the memory of a computer. Each different way is referred to as a *data structure*.

For instance, we experimented with lists and dictionnaries. Those data structures are native in python. 

We also experimented with object-oriented programming (OOP). We saw that lists could be reimplemented through OOP. OOP lets you implement whatever data structure you may need, way beyond those that are native in python.

One of the most useful data structure in Computer Science is that of [trees](https://en.wikipedia.org/wiki/Tree_%28data_structure%29). 

Trees are not native in python, but of course they have been implemented many times in many libraries. Let us experiment with one of them, before you make your own trees. 

First we install the library:

In [5]:
!pip install treelib

Defaulting to user installation because normal site-packages is not writeable


Next we import trees and their nodes:

In [6]:
from treelib import Tree

Now we build a tree and visualize it:

In [7]:
tree = Tree()
tree.create_node("Harry", "harry")  # root node
tree.create_node("Jane", "jane", parent="harry")
tree.create_node("Bill", "bill", parent="harry")
tree.create_node("Diane", "diane", parent="jane")
tree.create_node("Mary", "mary", parent="diane")
tree.create_node("Mark", "mark", parent="jane")
tree.show()

Harry
├── Bill
└── Jane
    ├── Diane
    │   └── Mary
    └── Mark



The *height* or *depth* of a tree is the length of its longest branch:

In [8]:
tree.depth()

3

Notice how trees are made of subtrees:

In [9]:
sub_t = tree.subtree('diane')
sub_t.show()

Diane
└── Mary



They might be manipulated in all sorts of ways:

In [10]:
tree.move_node('mary', 'harry')
tree.remove_node('bill')
tree.show()

Harry
├── Jane
│   ├── Diane
│   └── Mark
└── Mary



Most the trees you will encounter do not have this pretty `show()` method. Instead, you will typically use a command like this to visualize them:

In [11]:
import json
print(json.dumps(tree.to_json(), indent=4))

"{\"Harry\": {\"children\": [{\"Jane\": {\"children\": [\"Diane\", \"Mark\"]}}, \"Mary\"]}}"


**Exercise.**


1.   Define a class `MyTree` having attributes `name` and `children`, where `children` is to be a list of `MyTree`. 
2.   Provide the constructor method `__init__(...)`, which takes the name of the node as parameter.  
3.   Provide an `add_child(...)` method, so that if `p` and `c` are of type `MyTree`, then `p.add_child(c)` adds `c` within the children of `p`.
4.   Use your methods to build an example tree of depth 2, and work at a quick and dirty way to visualize it. 




## Answer:

In [64]:
class MyTree:
    def __init__(self, name):
        self.name = name
        self.children = []
        
    def add_child(self, child):
        self.children.append(child)
        
    def visualize(self, indentation=0):
        print(self.name)
        for child in self.children:
            print('\t'*indentation + " |-> ", end="")
            child.visualize(indentation+1)
    

In [66]:
parent = MyTree("parent")
child_a = MyTree("child_a")
child_b = MyTree("child_b")
child_aa = MyTree("child_aa")
child_ab = MyTree("child_ab")

child_a.add_child(child_aa)
child_a.add_child(child_ab)
parent.add_child(child_a)
parent.add_child(child_b)

parent.visualize()

parent
 |-> child_a
	 |-> child_aa
	 |-> child_ab
 |-> child_b


## Regular expressions

During the lecture we gave several plausible mathematical definitions of what a computer is. One of them was that of *finite automata*. We were able characterize the set of problems that they are able to solve: they recognize *regular languages* i.e. those expressed by *regular expressions*. 

Later, we came to the conclusion that finite automata are quite a limited model of computation, and came to prefer *Turing machines* instead. 

Still, finite automata are used everywhere in Computer Science:
*    in order to provide finitary desciption of computer systems, so as to prove properties about them (this is called *model checking*). 
*    in order to pattern match and manipulate strings efficiently, as expressed through regular expressions. 

The second use case is extremely common in practice. Let us play with it.  

First we import the corresponding library:

In [12]:
import re

Let us have a sample text:

In [13]:
text = "This is a string with term1, but it does not have the other term. Just term1, really."

We now test whether a pattern is present:

In [14]:
re.findall("term2",text)

[]

In [15]:
re.findall("term1",text)

['term1', 'term1']

Here is where the first occurence was found:

In [16]:
match=re.search("term1",text)
print(match)
print(match.start())
print(match.end())

<re.Match object; span=(22, 27), match='term1'>
22
27


**Exercice.** Informally describe an algorithm, that searches for a string, within another. Come to the conclusion that this will be quite a bit of work, and come to appreciate the power of the regular expression module. 

## Answer

Let say we search for string A in string B. Let say we have a pointer (or letter number) nA=0 and mA=0 in A and nB=0 in B.

Here is the algorithm:
 - If the mA-th letter of A matches with the nB-th letter of B, then we increment mA and nB.
 - If it doesn't match, then we increment nA, set nB=0, and set mA=nA.
 - If nB arrives at the end of B, then B is found in A starting at the nA-th letter of A.
 - If nA (or mA) arrives at the end of A, the B is not found in A.

On a daily basis, programmers need to do things such as:

In [17]:
email=input("Please type your email address :")
domain=(re.split("@",email))[1]
print("I see, so your provider is",domain,"...")

Please type your email address :joseph.2a.37.33@gmail.com
I see, so your provider is gmail.com ...


Let us create a function that will print out results of searching for different patterns within a single text:

In [18]:
def mfindall(patterns,text):
    '''
    Takes in a list of regex patterns
    Prints a list of all matches
    '''
    for pattern in patterns:
        print("Searching for :",pattern)
        print(re.findall(pattern,text))
        print()

*Repetitions.*
1.    A character followed by the meta-character `*` is allowed  to be repeated zero or more times. 
2.    Using `+` means the character must appear at least once. 
3.    Using `?` means the character must appear zero or one time. 
4.    Using `{m}` means the character must appear `m` times. 
5.    Using `{m,n}` means the character must appear `m` to `n` times. Leaving out `n`, as in  `{m,}` means the value appears at least `m` times, with no maximum.

When a match is encountered, python tries to make it as big as possible.
    
Try:

In [19]:
text = "sdsd..sssddd...sdddsddd...dsds...dsssss...sdddd"

patterns =     [ 'sd*',         # s followed by zero or more d's
                'sd+',          # s followed by one or more d's
                'sd?',          # s followed by zero or one d's
                'sd{3}',        # s followed by three d's
                'sd{2,3}',      # s followed by two to three d's
                ]

mfindall(patterns,text)

Searching for : sd*
['sd', 'sd', 's', 's', 'sddd', 'sddd', 'sddd', 'sd', 's', 's', 's', 's', 's', 's', 'sdddd']

Searching for : sd+
['sd', 'sd', 'sddd', 'sddd', 'sddd', 'sd', 'sdddd']

Searching for : sd?
['sd', 'sd', 's', 's', 'sd', 'sd', 'sd', 'sd', 's', 's', 's', 's', 's', 's', 'sd']

Searching for : sd{3}
['sddd', 'sddd', 'sddd', 'sddd']

Searching for : sd{2,3}
['sddd', 'sddd', 'sddd', 'sddd']



*Character sets.* `[...]` will match any single character in the square brackets.

In [20]:
text = 'sdsd..sssddd...sdddsddd...dsds...dsssss...sdddd'

patterns = [ '[sd]',    # either s or d
            's[sd]+']   # s followed by one or more s or d
            

mfindall(patterns,text)

Searching for : [sd]
['s', 'd', 's', 'd', 's', 's', 's', 'd', 'd', 'd', 's', 'd', 'd', 'd', 's', 'd', 'd', 'd', 'd', 's', 'd', 's', 'd', 's', 's', 's', 's', 's', 's', 'd', 'd', 'd', 'd']

Searching for : s[sd]+
['sdsd', 'sssddd', 'sdddsddd', 'sds', 'sssss', 'sdddd']



*Exclusion.* `[^...]` will match any single character not in the brackets.

In [21]:
text= 'This is a string! But it has punctuation. How can we remove it?'

In [22]:
l=re.findall('[^!.? ]+',text)
print(l)

['This', 'is', 'a', 'string', 'But', 'it', 'has', 'punctuation', 'How', 'can', 'we', 'remove', 'it']


Don't forget your functionnal programming skills:

In [23]:
from functools import reduce
reduce(lambda x,y: x+" "+y, l, "")

' This is a string But it has punctuation How can we remove it'

*Character Ranges.*
The format used is `[start-end]`. For instance `[a-f]` would return matches with any instance of letters between `a` and `f`.

In [24]:
text = 'This is an example sentence. Lets see if we can find some letters.'

patterns=     [ '[a-z]+',      # sequences of lower case letters
                '[A-Z]+',      # sequences of upper case letters
                '[a-zA-Z]+',   # sequences of lower or upper case letters
                '[A-Z][a-z]+'] # one upper case letter followed by lower case letters
                
mfindall(patterns,text)

Searching for : [a-z]+
['his', 'is', 'an', 'example', 'sentence', 'ets', 'see', 'if', 'we', 'can', 'find', 'some', 'letters']

Searching for : [A-Z]+
['T', 'L']

Searching for : [a-zA-Z]+
['This', 'is', 'an', 'example', 'sentence', 'Lets', 'see', 'if', 'we', 'can', 'find', 'some', 'letters']

Searching for : [A-Z][a-z]+
['This', 'Lets']



*Escape Codes.*


`^` refers to the beginning of the line. 

`$` refers to the end.

`.` refers to any character. `\.` refers to a dot. So does `[.]`.

Moreover:
<table border="1" class="docutils">
<colgroup>
<col width="14%" />
<col width="86%" />
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Code</th>
<th class="head">Meaning</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td><tt class="docutils literal"><span class="pre">\d</span></tt></td>
<td>a digit</td>
</tr>
<tr class="row-odd"><td><tt class="docutils literal"><span class="pre">\D</span></tt></td>
<td>a non-digit</td>
</tr>
<tr class="row-even"><td><tt class="docutils literal"><span class="pre">\s</span></tt></td>
<td>whitespace (tab, space, newline, etc.)</td>
</tr>
<tr class="row-odd"><td><tt class="docutils literal"><span class="pre">\S</span></tt></td>
<td>non-whitespace</td>
</tr>
<tr class="row-even"><td><tt class="docutils literal"><span class="pre">\w</span></tt></td>
<td>alphanumeric</td>
</tr>
<tr class="row-odd"><td><tt class="docutils literal"><span class="pre">\W</span></tt></td>
<td>non-alphanumeric</td>
</tr>
</tbody>
</table>

Escapes are indicated by prefixing the character with a backslash. Unfortunately, a backslash must itself be escaped in normal Python strings, and that results in expressions that are very difficult to read. Using raw strings, created by prefixing the literal value with `r`, eliminates this problem and maintains readability.

In [25]:
text = 'This is a string with some numbers 1233 and a symbol #hashtag'

patterns =      [ r'\d+', # sequence of digits
                r'\D+', # sequence of non-digits
                r'\s+', # sequence of whitespace
                r'\S+', # sequence of non-whitespace
                r'\w+', # alphanumeric characters
                r'\W+', # non-alphanumeric
                ]

mfindall(patterns,text)

Searching for : \d+
['1233']

Searching for : \D+
['This is a string with some numbers ', ' and a symbol #hashtag']

Searching for : \s+
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

Searching for : \S+
['This', 'is', 'a', 'string', 'with', 'some', 'numbers', '1233', 'and', 'a', 'symbol', '#hashtag']

Searching for : \w+
['This', 'is', 'a', 'string', 'with', 'some', 'numbers', '1233', 'and', 'a', 'symbol', 'hashtag']

Searching for : \W+
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' #']



Regular expressions are faster when compiled:

In [26]:
text = "To email Guido, try guido@python.org or the older address guido@google.com."
email = re.compile(r'\w+@\w+\.[a-z]{3}')
email.findall(text)

['guido@python.org', 'guido@google.com']

They provide fast mechanisms for find/replace:

In [27]:
email.sub("xxx@xxx.xxx", text)

'To email Guido, try xxx@xxx.xxx or the older address xxx@xxx.xxx.'

The above does not quite work however:

In [28]:
email.findall('barack.obama@whitehouse.gov')

['obama@whitehouse.gov']

**Exercise.** Write an `email2` regexp so it does not drop the first name in the above example.

## Answer:

In [124]:
email2 = re.compile(r'[\w+.+]+@\w+\.[a-z]{3}')
email2.findall('barack.obama@whitehouse.gov')

['barack.obama@whitehouse.gov']

**Exercise.** Write an `ip` regexp which matches IP addresses. Assume `ipv4` format.

## Answer:

In [199]:
ip = re.compile(r'(\d{1,3}\.){3}\d{1,3}')
text = "my IP adress is 129.154.20.34"
ip.search(text)

<re.Match object; span=(16, 29), match='129.154.20.34'>

*Groups.* Parentheses indicate groups to extract. We often want to extract their components rather than the full match:

In [193]:
email3 = re.compile(r'([\w.]+)@(\w+)\.([a-z]{3})')

In [127]:
text = "To email Guido, try guido@python.org or the older address guido@google.com."
email3.findall(text)

[('guido', 'python', 'org'), ('guido', 'google', 'com')]

If you'd rather see the full match:

In [128]:
email3.search(text).group()

'guido@python.org'

We can use previously extracted groups to match repetions of words:

In [129]:
text="nnabnnnnnnnnnnnnnnnnnnnababababababnnnnnnnnnaaaaabbbnnnnnnnnnnnnn"
myreg=re.compile(r'(ab)\1')
myreg.search(text)

<re.Match object; span=(23, 27), match='abab'>

Equivalently we could have written:

In [89]:
text="nnabnnnnnnnnnnnnnnnnnnnababababababnnnnnnnnnaaaaabbbnnnnnnnnnnnnn"
myreg=re.compile(r'(ab){2}')
myreg.search(text)

<re.Match object; span=(23, 27), match='abab'>

Finally, we can use `|` as an or operator, between expressions.

In [131]:
text="nnabnnnnnnnnnnnnnnnnnnnababababababnnnnnnnnnaaaaabbbnnnnnnnnnnnnn"
myreg=re.compile(r'((ab){2})|nabn')
myreg.search(text)

<re.Match object; span=(1, 5), match='nabn'>

**Exercise.** Extend the previous expression that matched `ipv4`, to also match `ipv6` addresses. 

In [200]:
ip2 = re.compile(r'(([0-9a-f]{1,4}\:)+[0-9a-f]{1,4})|(\d{1,3}\.){3}\d{1,3}')
text = 'my IP adress is 2001:0db8:3333:4444:5555:6666:7777:8888 and not 129.154.20.34'
ip2.search(text)

<re.Match object; span=(16, 55), match='2001:0db8:3333:4444:5555:6666:7777:8888'>

*References.* There are [many more things](https://docs.python.org/2/library/re.html#regular-expression-syntax) you can do with regular expressions.


## Parsing

Regular expressions describe regular languages. Regular languages were discussed, at the theoretical-level during the lectures. They are an interesting, but nevertheless simple class of languages. 

The aim of this section is to teach you how to describe more complicated languages, such as programming languages. Programming languages were again discussed at the theoretical-level during the lectures, in terms of their grammar and semantics. 

This section will deal with language grammars, and semantics, at the practical level. We will rely on a library called `Tatsu` in order to :
1.   Describe language grammars.
2.   Recognize whether a given text obeys the grammar.
3.   *Parse* the text according to the grammar. 
4.   Provide semantics to the language.
5.   Translate the language into another. 

When a computer parses a text according to a grammar, it creates a tree-like data structure in memory, that represents the text in a well-organized manner. It is the computer's way of "understanding the text" and "keeping it in mind in an orderly fashion".  

When we provide a semantics for the language, we are telling the computer what to make of that tree-like representation of the text. For instance, if the language is progamming language, the tree represents a text which is a program, which the computer ought to execute. So, the semantics tells the computer how to take actions according to the content of that tree.

So, let us install the `Tatsu` library. Its documentation is [here](https://tatsu.readthedocs.io/en/stable/).

By-the-way, there are many other such libraries, nicely reviewed [here](https://tomassetti.me/parsing-in-python/). Another mainstream choice could have been the `Lark` library. 

In [148]:
!pip install tatsu

Defaulting to user installation because normal site-packages is not writeable


Let us import the library so we can use it. We also take this opportunity to give a short nickname to some of the trees generated by `tatsu`. 

In [149]:
import tatsu
from tatsu.ast import AST

These two will come as handy in order to visualize some of trees:

In [150]:
from pprint import pprint
import json

Our first custom language will be a language of arithmetic expressions.

We must first describe its grammar. The grammar of will be described according some conventions... i.e. according to a "language to describe language grammars"! 

There are several such metalanguages. This one is called EBNF. It is quite standard, powerful, and looks a lot like the way we described grammars during the lectures:

In [151]:
grammar="""
@@grammar::CALC


start
    =
    expression $
    ;


expression
    =
    | expression '+' term
    | expression '-' term
    | term
    ;


term
    =
    | term '*' factor
    | term '/' factor
    | factor
    ;


factor
    =
    | '(' expression ')'
    | number
    ;


number
    =
    /\d+/
    ;
"""

Some comments are in order:
```
start
    =
    expression $
    ;
```
says that a text starts (`start`) with an expression and ends (`$`) there. 


Expressions may contain sums, so:
```
expression
    =
    | expression '+' term
    | expression '-' term
    | term
    ;
```
says an expression is an expression plus/minus a term, or just a term. 

Terms may contain products, so:
```
term
    =
    | term '*' factor
    | term '/' factor
    | factor
    ;
```
says an expression is an expression times/divided by a factor, or just a factor. 

Factors may contain numbers or bracketted expressions:
```
factor
    =
    | '(' expression ')'
    | number
    ;

number
    =
    /\d+/
    ;
```
Notice how numbers are described: just by a regular expression in between `/.../`.



Let us tell `tatsu` to create a parser for that grammar. 

In [152]:
parser = tatsu.compile(grammar)

This parser can be used to check whether some text obeys the language grammar.

For instance, this does not work because it is not within our language:

In [153]:
parser.parse("Hello")

FailedLeftRecursion: (1:1) infinite left recursion :
Hello
^
expression
start

But this works:

In [156]:
parser.parse("3")

'3'

In [157]:
parser.parse("3+2")

('3', '+', '2')

In [158]:
parser.parse("(3+2)*4")

(('(', ('3', '+', '2'), ')'), '*', '4')

Let us parse a more complicated expression, and visualize the resulting tree-like structure in a nicer way:

In [159]:
ast = parser.parse("5 * (1-2) + 50 * ( 10 - 20 )")
print(type(ast))
pprint(ast, width=20, indent=4)

<class 'tuple'>
(   (   '5',
        '*',
        (   '(',
            (   '1',
                '-',
                '2'),
            ')')),
    '+',
    (   '50',
        '*',
        (   '(',
            (   '10',
                '-',
                '20'),
            ')')))


We see that the parser is indeed returning a tree-like structure.
*   The root of the tree is `+`
*   It has too branches whose roots are `*`
*   Those have leafs `3` and `5`, but also more branches, etc.

Tree-like representations of languages as generated by parsers are referred to as *abstract syntax trees*---which is why we chose to call our variable `ast`.

Technically this particular `ast` is just a list, though. 
We may wish for a more informative tree-like structure, for instance one that tags `op` in front of an operation whenever it finds one. This is what the following, annotated grammar will produce:



In [160]:
grammar="""
@@grammar::CALC


start
    =
    expression $
    ;


expression
    =
    | left:expression op:'+' right:term
    | left:expression op:'-' right:term
    | term
    ;


term
    =
    | left:term op:'*' right:factor
    | left:term '/' right:factor
    | factor
    ;


factor
    =
    | '(' @:expression ')'
    | number
    ;


number
    =
    /\d+/
    ;

"""

Some comments: 
```
op:'*'
```
will tag the occurrence of `*` with `op`. 
```
@:
```
will skip the surrounding parentheses in the abstract syntax tree production, resulting in a simpler, more readable tree. 

Indeed, notice that parentheses are only useful in the text version of the expressions, in order to lift ambiguities. But the tree-like structure has lifted those ambiguities already, so it does not need to hold cumbersome parentheses.

Notice that the resulting abstract syntax tree is no longer a list:

In [161]:
parser = tatsu.compile(grammar)
ast = parser.parse("5 * (1-2) + 50 * ( 10 - 20 )")
print(type(ast))
pprint(ast, width=20, indent=4)

<class 'tatsu.ast.AST'>
{'left': {'left': '5', 'op': '*', 'right': {'left': '1', 'op': '-', 'right': '2'}}, 'op': '+', 'right': {'left': '50', 'op': '*', 'right': {'left': '10', 'op': '-', 'right': '20'}}}


This object can be navigated through its attributes:

In [162]:
ast.op

'+'

In [163]:
ast.left

{'left': '5', 'op': '*', 'right': {'left': '1', 'op': '-', 'right': '2'}}

In [164]:
ast.left.op

'*'

**Exercise.** Navigate to the `10` that way.

## Answer

In [168]:
ast.right.right.left

'10'

So, we can parse arithmetic expressions now. 

Let us give them meaning, i.e. a semantics that says how to evaluate them. 

In [171]:
class CalcBasicSemantics(object):
    def number(self, ast):
        print("number",ast)
        return int(ast)

    def term(self, ast):
        print("term",ast)
        if not isinstance(ast, AST):
            return ast
        elif ast.op == '*':
            return ast.left * ast.right
        elif ast.op == '/':
            return ast.left / ast.right
        else:
            raise Exception('Unknown operator', ast.op)

    def expression(self, ast):
        print("expression",ast)
        if  not isinstance(ast, AST):
            return ast
        elif ast.op == '+':
            return ast.left + ast.right
        elif ast.op == '-':
            return ast.left - ast.right
        else:
            raise Exception('Unknown operator', ast.op)

In [172]:
ast=parser.parse("3 + 5 * ( 10 - 20 )")
pprint(ast, width=20, indent=4)
result = parser.parse("3 + 5 * ( 10 - 20 )",semantics=CalcBasicSemantics())
print("-------------")
print(result)


{'left': '3', 'op': '+', 'right': {'left': '5', 'op': '*', 'right': {'left': '10', 'op': '-', 'right': '20'}}}
number 3
term 3
term 3
expression 3
number 5
term 5
number 10
term 10
term 10
expression 10
number 20
term 20
term 20
expression {'left': 10, 'op': '-', 'right': 20}
expression 10
term {'left': 5, 'op': '*', 'right': -10}
term 5
expression {'left': 3, 'op': '+', 'right': -50}
expression 3
-------------
-47


**Exercise.** 
1.   Explain the order in which the prints get executed. Which [depth-first tree traversal order](https://en.wikipedia.org/wiki/Tree_traversal) is used? Why this choice? 
2.   Try without the `if not isinstance(ast,AST)`. Explain why it was there in the first place. 



## Answer

1) We choose to traverse in a Depth-first search, i.e. a recursive manner, first to the left and then to the right. The reason is to "collapse" expression recursivly, because we can only apply operations to number and not operations.

2) if we remove `if not isinstance(ast,AST)`, we get an error "`'int' object has no attribute 'op'`", because when `ast` is a number, we want to return it as is, as it is not an expression to be computed (it has no `op` attribute).

These `isinstance(...)` and `elif ast.op==...` are not very nice, they make our semantics rather cumbersome. 

It turns out we can get rid of them, at the cost of a more fined-grained grammar:

In [ ]:
grammar ="""
@@grammar::CALC

start
    =
    expression $
    ;


expression
    =
    | addition
    | subtraction
    | term
    ;


addition
    =
    left:expression op:'+' ~ right:term
    ;

subtraction
    =
    left:expression op:'-' ~ right:term
    ;


term
    =
    | multiplication
    | division
    | factor
    ;


multiplication
    =
    left:term op:'*' ~ right:factor
    ;


division
    =
    left:term '/' ~ right:factor
    ;


factor
    =
    | '(' ~ @:expression ')'
    | number
    ;


number
    =
    /\d+/
    ;
"""

In [ ]:
class CalcSemantics(object):
    def number(self, ast):
        print("number",ast)
        return int(ast)

    def addition(self, ast):
        print("addition",ast)
        return ast.left + ast.right

    def subtraction(self, ast):
        print("subtraction",ast)
        return ast.left - ast.right

    def multiplication(self, ast):
        print("multiplication",ast)
        return ast.left * ast.right

    def division(self, ast):
        print("division",ast)
        return ast.left / ast.right

In [ ]:
parser = tatsu.compile(grammar)
ast=parser.parse("3 + 5 * ( 10 - 20 )")
pprint(ast, width=20, indent=4)
result= parser.parse("3 + 5 * ( 10 - 20 )",semantics=CalcSemantics())
print("-------------")
print(result)

{   'left': '3',
    'op': '+',
    'right': {   'left': '5',
                 'op': '*',
                 'right': {   'left': '10',
                              'op': '-',
                              'right': '20'}}}
number 3
number 5
number 10
number 20
subtraction AST({'left': 10, 'op': '-', 'right': 20})
multiplication AST({'left': 5, 'op': '*', 'right': -10})
number 5
addition AST({'left': 3, 'op': '+', 'right': -50})
-------------
-47


**Exercise.**  Explain how this solution manages to avoid `if` statements in the semantics. 

## Answer

This solutions avoids `if` statements in the semantics, by creating a separate gramatical class for each operator/entity so the semantics uses different functions for different operators.

At this stage, the parser is generating an abstract syntax tree whose type is an object of type the `tatsu` pre-defined class AST: 

In [ ]:
type(ast)

Their branches are themselves AST:

In [ ]:
type(ast.right)

This is not bad, but we may prefer to have our custom objects forming the tree instead. For instance, this could be used as a way to retain the type of subexpression at hand, e.g. whether an addition, a multiplication...

Further annotations in the grammar let you do this. Below, `::Add` specifies that a node of type `Add` should be used to represent that subexpression:

In [349]:
grammar = """
@@grammar::Calc


start
    =
    expression $
    ;


expression
    =
    | addition
    | subtraction
    | term
    ;


addition::Add
    =
    left:term op:'+' ~ right:expression
    ;


subtraction::Subtract
    =
    left:term op:'-' ~ right:expression
    ;


term
    =
    | exponentiation
    | multiplication
    | division
    | factor
    ;

exponentiation::Exponentiate
    =
    left:factor op:'**' ~ right:term
    ;
    
multiplication::Multiply
    =
    left:factor op:'*' ~ right:term
    ;


division::Divide
    =
    left:factor '/' ~ right:term
    ;


factor
    =
    | subexpression
    | number
    ;


subexpression
    =
    '(' ~ @:expression ')'
    ;


number::int
    =
    /\d+/
    ;
"""

In [350]:
parser = tatsu.compile(grammar, asmodel=True)
ast = parser.parse("3 + 5 * ( 10 - 20 ) ** 2")
print(type(ast).__name__)
print(type(ast))
print(json.dumps(ast.asjson(), indent=4))


Add
<class 'tatsu.synth.Add'>
{
    "__class__": "Add",
    "left": 3,
    "right": {
        "__class__": "Multiply",
        "left": 5,
        "right": {
            "__class__": "Exponentiate",
            "left": {
                "__class__": "Subtract",
                "left": 10,
                "right": 20,
                "op": "-"
            },
            "right": 2,
            "op": "**"
        },
        "op": "*"
    },
    "op": "+"
}


Instead of directly evaluating our expressions as we would parse them, all-in-one-go as we did before, it is better practice to produce the abstract syntax tree, and then walk through the abstract syntax tree.

One of the reasons for that is that the abstract syntax tree may require several traversals before being evaluated, e.g. 
for 
1.    *name analysis* i.e. understanding which occurence of a variable corresponds to which declaration.
2.    *type checking* as in the lectures.
3.    *linking* i.e. relating different modules.
4.    *optimization* i.e. transforming the code to make it more efficient.

A convenient way to walk through the abstract syntax tree is to use a Walker, i.e. an object that you define and that inherits from `tatsu.walkers.NodeWalker`.

In [351]:
from tatsu.walkers import NodeWalker

In [352]:
class CalcWalker(NodeWalker):
    def walk_object(self, node):
        print("Object", node)
        return node

    def walk__add(self, node):
        print("Add")
        return self.walk(node.left) + self.walk(node.right)

    def walk__subtract(self, node):
        print("Subtract")
        return self.walk(node.left) - self.walk(node.right)

    def walk__multiply(self, node):
        print("Multiply")
        return self.walk(node.left) * self.walk(node.right)

    def walk__divide(self, node):
        print("Divide", node)
        return self.walk(node.left) / self.walk(node.right)
    
    def walk__exponentiate(self, node):
        print("Exponentiate", node)
        return self.walk(node.left) ** self.walk(node.right)

In [353]:
parser = tatsu.compile(grammar, asmodel=True)
ast = parser.parse("3 + 5 * ( 10 - 20 ) ** 2")
print(json.dumps(ast.asjson(), indent=4))
result = CalcWalker().walk(ast)
print("-------------")
print(result)

{
    "__class__": "Add",
    "left": 3,
    "right": {
        "__class__": "Multiply",
        "left": 5,
        "right": {
            "__class__": "Exponentiate",
            "left": {
                "__class__": "Subtract",
                "left": 10,
                "right": 20,
                "op": "-"
            },
            "right": 2,
            "op": "**"
        },
        "op": "*"
    },
    "op": "+"
}
Add
Object 3
Multiply
Object 5
Exponentiate {
  "__class__": "Exponentiate",
  "left": {
    "__class__": "Subtract",
    "left": 10,
    "right": 20,
    "op": "-"
  },
  "right": 2,
  "op": "**"
}
Subtract
Object 10
Object 20
Object 2
-------------
503


**Exercise.** 
Explain the order in which the prints get executed. Which [depth-first tree traversal order](https://en.wikipedia.org/wiki/Tree_traversal) is used? How does it still manage to get to a result?

## Answer

Here the order is "recursive", it still works because the parsing follows the parenthesis for speraration.

So far the semantics that we have used were aiming at executing the expressions down to a value. 

We may, instead, want to provide a *translation semantics*, i.e. produce a translation into another language instead. 

For instance, this is what compilers do: they translate human-readable computer languages, into assembly-language, which then further gets translated machine language, i.e. zeros and ones that is the computer-readable and executable language. 

`tatsu` helps you write template-based code generators.

The example below translates arithmetic expressions into posfix notation.

In [354]:
from tatsu.codegen import ModelRenderer
from tatsu.codegen import CodeGenerator
import sys

THIS_MODULE =  sys.modules[__name__]


class PostfixCodeGenerator(CodeGenerator):
    def __init__(self):
        super().__init__(modules=[THIS_MODULE])


class Number(ModelRenderer):
    template = '''\
    PUSH {value}'''


class Add(ModelRenderer):
    template = '''\
    {left}
    {right}
    ADD'''


class Subtract(ModelRenderer):
    template = '''\
    {left}
    {right}
    SUB'''


class Multiply(ModelRenderer):
    template = '''\
    {left}
    {right}
    MUL'''


class Divide(ModelRenderer):
    template = '''\
    {left}
    {right}
    DIV'''

class Power(ModelRenderer):
    template = '''\
    {left}
    {right}
    DIV'''

In [355]:
parser = tatsu.compile(grammar, asmodel=True)
ast = parser.parse("3 + 5 * ( 10 - 20 )")
print(json.dumps(ast.asjson(), indent=4))
postfix = PostfixCodeGenerator().render(ast)
print(postfix)

{
    "__class__": "Add",
    "left": 3,
    "right": {
        "__class__": "Multiply",
        "left": 5,
        "right": {
            "__class__": "Subtract",
            "left": 10,
            "right": 20,
            "op": "-"
        },
        "op": "*"
    },
    "op": "+"
}
3
5
10
20
SUB
MUL
ADD


## Mini-project



1.   By incremental modifications of the last version of the above-given grammars, provide a grammar for `Mini-ML` language we saw in class.
2.   Implement the big steps semantics of `Mini-ML` we saw in class. 

*Advanced.*

3.   In your grammar and semantics, modify the type of constants to be booleans instead of integers or floats, and let the primitives be Not, Or, And, Nand gates, so as to eventually obtain a custom language: `Circuit-ML`. 
4.    Implement a type-checker for `Circuit-ML`.


## Answser Q1

In [362]:
grammar = """
@@grammar::Calc


start
    =
    expression $
    ;


expression
    =
    | operations
    | pair
    | function
    | term 
    ;




pair
    =
    left:expression op:',' ~ right:expression
    ;
    
function
    =
    op:'lambda' left:term ':' ~ right:expression
    ;
    


term
    =
    | identifier
    | constant
    | subexpression
    ;
    
identifier
    =
    /[a-zA-Z]+/
    ;

subexpression
    =
    '(' ~ @:expression ')'
    ;



constant
    =
    | bool
    | number
    ;
    
bool
    =
    /[tf]/
    ;
    
number
    =
    /\d+/
    ;

    
    


operations
    =
    | assignment
    | add
    | substract
    | multiply
    | divide
    | exponentiate
    | apply
    ;
    

assignment
    =
    left:term op:'=' ~ right:expression
    ;

add
    =
    left:term op:'+' ~ right:expression
    ;

substract
    =
    left:term op:'-' ~ right:expression
    ;

multiply
    =
    left:term op:'*' ~ right:expression
    ;

divide
    =
    left:term '/' ~ right:expression
    ;
    
exponentiate
    =
    left:term op:'**' ~ right:expression
    ;
    
apply
    =
    left:term op:':' right:expression
    ;
"""

In [363]:
parser = tatsu.compile(grammar, asmodel=True)
ast = parser.parse("lambda a : ((a = 2 * 2 + a : b), (2 * 3))")
print(ast)
print(json.dumps(ast.asjson(), indent=4))

{'op': 'lambda', 'left': 'a', 'right': {'left': {'left': 'a', 'op': '=', 'right': {'left': '2', 'op': '*', 'right': {'left': '2', 'op': '+', 'right': {'left': 'a', 'op': ':', 'right': 'b'}}}}, 'op': ',', 'right': {'left': '2', 'op': '*', 'right': '3'}}}
{
    "op": "lambda",
    "left": "a",
    "right": {
        "left": {
            "left": "a",
            "op": "=",
            "right": {
                "left": "2",
                "op": "*",
                "right": {
                    "left": "2",
                    "op": "+",
                    "right": {
                        "left": "a",
                        "op": ":",
                        "right": "b"
                    }
                }
            }
        },
        "op": ",",
        "right": {
            "left": "2",
            "op": "*",
            "right": "3"
        }
    }
}


## Some solutions

In [358]:
class MyTree:

    def __init__(self,n="",ch=None):
        if ch is None:
            ch = []
        self.name = n
        self.children = ch



    def add_child(self,c):
        if c not in self.children:
            self.children.append(c)



    def __repr__(self,ind=0):
        s = ""
        for _ in range(ind):
            s += " "
        s += self.name
        if self.children:
            s += ":"
            for c in self.children:
                s += "\n"
                for _ in range(ind):
                    s += " "
                s += c.__repr__(ind+1)
        return s

toto = MyTree("toto")
toto.add_child(MyTree("tutu"))
toto.add_child(MyTree("tata"))
bobo = MyTree("bobo",[toto,MyTree("jojo")])
print(bobo)

bobo:
 toto:
   tutu
   tata
 jojo


In [359]:
ipv4pat = r"\d{0,3}\.\d{0,3}\.\d{0,3}\.\d{0,3}"

In [360]:
ipv6pat = r"([0-9a-fA-F]{0,4}:){7}[0-9a-fA-F]{0,4}"

In [361]:
ippat = re.compile("(" + ipv4pat + r")|(" + ipv6pat + ")")

Post-order (LRN). The philosophy is to evaluate leafs before we can act with operations upon them. 